In [1]:
from src.RawDataLoaders import CAsT_RawDataLoader
from src.models_and_transforms.text_transforms import Rewriter_Query_Resolver_Transform, Rewriter_Context_Query_Merge_Transform, \
                                                      BART_Numericalise_Transform, Rewriter_Context_Target_Transform
from src.pipe_datasets import CAsT_Query_ReWriting_Dataset
from src.models_and_transforms.BART_models import BART_Query_ReWriter
from src.trainers import Model_Trainer

from transformers import BertTokenizer, BartTokenizer
from pytorch_lightning import Trainer, Callback, seed_everything
from pytorch_lightning.loggers import WandbLogger
import torch
from tqdm.auto import tqdm 
import  random
import os
import numpy as np
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
seed_everything(42)

42

In [3]:
raw_data_loader = CAsT_RawDataLoader()
get_query_fn = raw_data_loader.get_query
get_doc_fn = raw_data_loader.get_doc

In [14]:
train_raw_samples = raw_data_loader.get_topics("train", ignore_missing_q_rels=True)

In [15]:
len(train_raw_samples)

376

In [16]:
train_samples = Rewriter_Query_Resolver_Transform(get_query_fn)(train_raw_samples)
train_samples = Rewriter_Context_Query_Merge_Transform()(train_samples)
train_samples = Rewriter_Context_Target_Transform()(train_samples)
train_samples = BART_Numericalise_Transform(fields=[('input_text', 'input_ids'), ('target_text', 'target_ids')])(train_samples)

In [17]:
train_dataset = CAsT_Query_ReWriting_Dataset(train_samples, get_query_fn, merge_mode="last_turn_rewrite")

In [19]:
train_dataset[4]

{'prev_turns': ['31_1', '31_2', '31_3', '31_4'],
 'q_id': '31_5',
 'q_rel': ['MARCO_2509296'],
 'unresolved_query': 'Can it spread to the throat?',
 'previous_queries': ['What is throat cancer?',
  'Is throat cancer treatable?',
  'Tell me about lung cancer.',
  "What are lung cancer's symptoms?"],
 'resolved_query': 'Can lung cancer spread to the throat?',
 'input_text': "What is throat cancer? Is throat cancer treatable? Tell me about lung cancer. What are lung cancer's symptoms? query: Can it spread to the throat?",
 'target_text': 'Can lung cancer spread to the throat?',
 'input_ids': [0,
  2264,
  16,
  14599,
  1668,
  116,
  1534,
  14599,
  1668,
  3951,
  868,
  116,
  11378,
  162,
  59,
  10665,
  1668,
  4,
  653,
  32,
  10665,
  1668,
  18,
  5298,
  116,
  25860,
  35,
  2615,
  24,
  2504,
  7,
  5,
  14599,
  116,
  2],
 'target_ids': [0, 10836, 10665, 1668, 2504, 7, 5, 14599, 116, 2]}

In [20]:
train_dataloader = train_dataset.to_dataloader(16)

In [9]:
batch = list(iter(train_dataloader))[17]

In [10]:
batch

{'input_ids': tensor([[    0, 35438,   162,  ...,     1,     1,     1],
         [    0, 35438,   162,  ...,     1,     1,     1],
         [    0, 25860,    35,  ...,     1,     1,     1],
         ...,
         [    0,  2264,    16,  ...,     1,     1,     1],
         [    0,  2264,    16,  ...,     1,     1,     1],
         [    0,  2264,    16,  ...,     1,     1,     1]]),
 'input_attention_mask': tensor([[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.]]),
 'decoder_input_ids': tensor([[    0, 35438,   162,  ...,     1,     1,     1],
         [    0, 35438,   162,  ...,     1,     1,     1],
         [    0, 25860,    35,  ...,     1,     1,     1],
         ...,
         [    0,  2264,    16,  ...,     1,     1,     1],
         [    0,  2264,    16,  ...,     1,     1,     1],
 

In [23]:
val_raw_samples = raw_data_loader.get_topics("dev", ignore_missing_q_rels=True)
val_dataset = CAsT_Query_ReWriting_Dataset(val_raw_samples, get_query_fn, merge_mode="last_turn_rewrite")
val_dataloader = val_dataset.to_dataloader(32)

In [22]:
len(val_dataset)

103

In [24]:
model = BART_Query_ReWriter()

Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
trainer = Trainer(gpus=1, gradient_clip_val=0.5, check_val_every_n_epoch=0.5)
trainer.fit(model, train_dataloader, val_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name | Type                         | Params
------------------------------------------------------
0 | BART | BartForConditionalGeneration | 406 M 


{'BLEU': 0.2362831067985046}
-----EXAMPLE------
Input Text: ' query: What are the pros and cons of electric cars?'
      Pred: 'What are the pros and cons of electric cars? CNN��road 'ense rev during the week of Maybel "anga among the cast members of "Cind sw in "'
    Target: 'What are the pros and cons of electric cars?'
------------------


{'BLEU': 0.24431057377978074}
-----EXAMPLE------
Input Text: 'Why do turkey and Turkey share the same name? Where are turkeys from? query: What was their importance in native cultures?'
      Pred: 'What was turkeys' importance in native cultures in 1800 rest?",road " rev'during the 1930s and '40s? What was the importance of turkeys in native culture in 1800s?'
    Target: 'What was the importance of turkey in native cultures?'
------------------


{'BLEU': 0.24864010061382458}
-----EXAMPLE------
Input Text: 'What is taught in sociology? What is the main contribution of Auguste Comte to sociology? What is the role of positivism in sociology? What is Herbert Spencer known for in sociology? How is Herbert Spencer's work related to Comte? What is the functionalist theory in sociology? query: What is its main criticism?'
      Pred: 'What is the main criticism of the functionalist theory in sociology? The Rep415road "sw eating don in rev during the week" among members of the "c swarm among the cast'
    Target: 'What is the main criticism of functionalist theory in sociology?'
------------------


{'BLEU': 0.25202663803217507}
-----EXAMPLE------
Input Text: 'Who is the most successful pirate of all time? Who is the most famous female pirate? Tell me more about Ching Shih. What were Ching Shih's code of laws? query: How did she die?'
      Pred: 'How did Ching Shih of the pirate past die? CNNroad415'holiday rev " eatingame during the pirate era" "Bic 'c all in 'b sw among the pirates of past?"'
    Target: 'How did Ching Shih die?'
------------------


...

{'BLEU': 0.25066067452613494}
-----EXAMPLE------
Input Text: 'What do Spanish people do on Christmas day? What do Spanish people eat for Christmas dinner? What do Spanish people eat on Christmas eve? What is the Spanish Christmas Lottery? How does the Spanish Christmas Lottery drawing work? How are the Spanish Christmas Lottery tickets sold? What is Tió de Nadal? How did the tradition of Tió de Nadal start? query: How do they celebrate Three Kings Day?'
      Pred: 'How do Spanish people celebrate Three Kings Day? The answer may surprise the Spanish people. The Christmas Lottery and Tió de Nadal are just some of the traditions Spanish people observe on Christmas Day and Christmas Eve and on Christmas Eve. The Spanish people eat for Christmas dinner and they eat on Christmas eve.'
    Target: 'How do Spanish people celebrate Three Kings Day?'
------------------


{'BLEU': 0.2456879505593709}
-----EXAMPLE------
Input Text: 'What are mammals? What are mammals' key characteristics? query: How many legs do they have?'
      Pred: 'How many legs do mammals in the human world have? The�� Rep415 "emroad 'ense rev during the springendWeekanga lights'
    Target: 'How many legs do mammals have?'
------------------


{'BLEU': 0.2529478360542443}
-----EXAMPLE------
Input Text: 'What are the pros and cons of electric cars? Tell me more about Tesla the car company. Why did Elon Musk start Tesla? What Tesla car models are available? What is the best selling Tesla car model? What is unique about the Tesla Model 3? What are the safety features of Tesla Model 3? Why is Tesla building Gigafactories? What makes the Tesla batteries unique? query: How long can they last?'
      Pred: 'How long can the Tesla batteries last? CNN Rep415road "g 'ame eating rev during the week" "gast 'barm all sw in "'
    Target: 'How long can Tesla car batteries last?'
------------------


{'BLEU': 0.2506350466354144}
-----EXAMPLE------
Input Text: 'What's the difference between soup and stew? Is chilli a stew? Is goulash a stew? What are popular stews in France? query: How is cassoulet made?'
      Pred: 'How is cassoulet made? CNN��road415'holiday st revame eating "st stews in France "goulash and cassoulets being made?"'
    Target: 'How is cassoulet made?'
------------------



1

In [ ]:
torch.save(model.save)

In [13]:
my_trainer = Model_Trainer(gpus=[3])
nan_batch = my_trainer.train(model, train_dataloader)

Detected 8 GPUS available, using [3].



Keyboard Interrupt!


In [16]:
%debug

> /nfs/phd_by_carlos/notebooks/src/models_and_transforms/BART_models.py(59)validation_epoch_end()
     57         samples = []
     58         for batch in  outputs:
---> 59             for i in range(len(batch["input_ids"])):
     60                 sample_obj = {}
     61                 sample_obj["generated_ids"] = batch["generated_ids"][i].tolist()



ipdb>  batch["input_ids"]


*** TypeError: 'NoneType' object is not subscriptable


ipdb>  batch
ipdb>  outputs


[None]


ipdb>  exit


In [22]:
gen = model.generate(batch['input_ids'].to(device), num_beams=4, max_length=40, early_stopping=True)

In [23]:
gen.tolist()

[[2,
  0,
  2264,
  16,
  14599,
  1668,
  116,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 [2,
  0,
  6209,
  14599,
  1668,
  3951,
  868,
  116,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 [2,
  0,
  35438,
  162,
  59,
  10665,
  1668,
  4,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 [2,
  0,
  2264,
  32,
  10665,
  1668,
  18,
  5298,
  116,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 [2,
  0,
  10836,
  10665,
  1668,
  2504,
  7,
  5,
  14599,
  116,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


## Debugging code

In [15]:
batch=list(iter(train_dataloader))[0]

In [23]:
model.cuda().BART.model(batch['input_ids'].cuda(), decoder_input_ids=batch['decoder_input_ids'].cuda())

(tensor([[[nan, nan, nan,  ..., nan, nan, nan]],
 
         [[nan, nan, nan,  ..., nan, nan, nan]]], device='cuda:0',
        grad_fn=<TransposeBackward0>),
 ((tensor([[[nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan],
            ...,
            [nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan]],
   
           [[nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan],
            ...,
            [nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan]]], device='cuda:0',
          grad_fn=<TransposeBackward0>),
   None),
  [{'self': {'prev_key': tensor([[[[-1.8211,  5.3233, -2.7674,  ..., -2.6931, -7.5947,  0.3828]],
     
              [[-2.5837, -2.78

In [10]:
model.cpu().BART.model(batch['input_ids'].cpu(), decoder_input_ids=batch['decoder_input_ids'].cpu())

(tensor([[[ 0.6262,  0.3487, -1.3632,  ...,  0.7484,  0.4604,  0.7461]],
 
         [[-1.2074,  0.7323,  0.3149,  ...,  0.0720,  0.2123,  1.6422]]],
        grad_fn=<TransposeBackward0>),
 ((tensor([[[ 0.0100,  0.0266,  0.0208,  ..., -0.0044, -0.0085, -0.0043],
            [-0.0054, -0.3839, -0.2030,  ..., -0.1332,  0.0723, -0.0573],
            [ 0.1947, -0.3128,  0.1877,  ..., -0.1642,  0.0895,  0.0182],
            ...,
            [ 0.0501, -0.5405,  0.2048,  ..., -0.0651, -0.0487, -0.2173],
            [ 0.1669, -0.2895,  0.0097,  ..., -0.1521, -0.0590, -0.1472],
            [ 0.1197,  0.1636,  0.0273,  ..., -0.0256, -0.1019,  0.0898]],
   
           [[-0.0042,  0.0110,  0.0035,  ..., -0.0022, -0.0021,  0.0008],
            [ 0.0591, -0.0743, -0.4011,  ..., -0.1129,  0.0362, -0.1217],
            [-0.1800,  0.1222, -0.0420,  ..., -0.1162,  0.1571, -0.0345],
            ...,
            [-0.2487, -0.1782,  0.0875,  ..., -0.0217,  0.0142,  0.0182],
            [-0.2688, -0.1255,  0

In [ ]:
model.BART.

In [29]:
dataloader_list = list(iter(train_dataloader))

In [31]:
model.to("cuda:3")
for idx, batch in enumerate(dataloader_list):
    model.eval()
    for key in list(batch.keys()):
        batch[key] = batch[key].to("cuda:3")
    loss = model.training_step(batch,0)["loss"]
    print(loss)
    if torch.isnan(loss):
        print(f"found nan at idx {idx}")
        break

tensor(6.2256, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(4.3583, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(7.0870, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(4.4793, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(5.8142, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(5.9656, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(4.8488, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(6.1428, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(5.3876, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(7.2418, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(3.0531, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(5.3417, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(5.7088, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(4.6437, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(3.9586, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(6.0022, device='cuda:3', grad_fn=<NllLossBackward>)
tensor(4.0564, device='cuda:3', grad_fn=<NllLossBackward

In [20]:
device = "cuda"
model.to(device)
for key in list(batch.keys()):
    batch[key] = batch[key].to(device)

model.training_step(batch,0)["loss"]

tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)

In [ ]:
batch['decoder_input_ids'].shape

In [44]:
# model.cpu()
model.training_step(batch,0)

{'loss': tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)}

In [25]:
for p in model.parameters():
    print(torch.isnan(p).any())

tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(F